## Importing necessary packages and data

In [1]:
import numpy as np
import pandas as pd
from datetime import date

## 1. Forward Pricing Fundamentals

### Data

Consider a Treasury note with the following characteristics:

| Field | Value |
|-------|-------|
| Coupon | 4.00% |
| Maturity | 2030-02-28 |
| Issue Date | 2023-02-28 |
| First Coupon | 2023-08-31 |

Market conditions as of **2023-04-18**:

| Field | Value |
|-------|-------|
| Spot Price | 102-02 (i.e., 102 + 2/32) |
| Repo Rate | 4.85% |
| Forward Date | 2023-08-01 |

### 1.1 Accrued Interest

To calculate the accrued interest $A_t$ and $A_T$, we must adhere to the Actual/Actual day count convention used for US Treasuries.

**Key Dates:**
- **Current Date ($t$):** 2023-04-18
- **Forward Date ($T$):** 2023-08-01

**Reference Period:**  
The coupon period runs from the Issue Date (**2023-02-28**) to the First Coupon Date (**2023-08-31**).

**Formula for accrued interest:**  
$$
A = \frac{C * N}{2} \times \frac{\text{Days since last coupon}}{\text{Days in current coupon period}}
$$

In [2]:
# Inputs
face_value = 100
coupon_rate = 0.04
issue_date = date(2023, 2, 28)
first_coupon_date = date(2023, 8, 31)
current_date = date(2023, 4, 18)  # t
forward_date = date(2023, 8, 1)   # T

# Helper functions for day counts
def days_between(d1, d2):
    return (d2 - d1).days

# Accrued Interest Calculation
# Coupon period days
days_in_period = days_between(issue_date, first_coupon_date)

# Accrued at t (Current Date)
days_accrued_t = days_between(issue_date, current_date)
accrued_t = ((coupon_rate * face_value) / 2) * (days_accrued_t / days_in_period)

# Accrued at T (Forward Date)
days_accrued_T = days_between(issue_date, forward_date)
accrued_T = ((coupon_rate * face_value) / 2) * (days_accrued_T / days_in_period)

# Display the accrued interest data
accrued_data = {
    "Description": [
        "Days in coupon period (Feb 28 - Aug 31)",
        "Days accrued at t (Feb 28 - Apr 18)",
        "Accrued Interest at t (A_t)",
        "Days accrued at T (Feb 28 - Aug 01)",
        "Accrued Interest at T (A_T)"
    ],
    "Value": [
        days_in_period,
        days_accrued_t,
        f"{accrued_t:.6f}",
        days_accrued_T,
        f"{accrued_T:.6f}"
    ]
}

df_accrued = pd.DataFrame(accrued_data)
df_accrued

,Description,Value
0,Days in coupon period (Feb 28 - Aug 31),184
1,Days accrued at t (Feb 28 - Apr 18),49
2,Accrued Interest at t (A_t),0.532609
3,Days accrued at T (Feb 28 - Aug 01),154
4,Accrued Interest at T (A_T),1.673913


### 1.2 Forward Price (No Interim Coupon)

We calculate the forward price for delivery at $T$. The spot purchase must be financed at the repo rate until delivery.

**Formula:**

$$
F(t, T) = (P_t + A_t)\left[1 + r_{\text{repo}} \cdot \tau_{\text{ACT/360}}\right] - A_T
$$

where:

- $P_t$: Clean spot price ($102\text{-}02$, or $102 + 2/32$)
- $r_{\text{repo}}$: $4.85\%$
- $\tau_{\text{ACT/360}}$: Day count fraction using Actual/360 convention

In [3]:
# Inputs
spot_price_32nds = 102 + 2/32
repo_rate = 0.0485

# Forward Price Calculation
# Dirty Price at t
dirty_price_t = spot_price_32nds + accrued_t

# Time fraction tau (Actual/360)
days_t_to_T = days_between(current_date, forward_date)
tau_act360 = days_t_to_T / 360.0

# Forward Price Formula
# Future Value of Dirty Price - Accrued Interest at T
fv_dirty_price = dirty_price_t * (1 + repo_rate * tau_act360)
forward_clean_price = fv_dirty_price - accrued_T

# Display the forward price data
forward_price_data = {
    "Description": [
        "Forward Price"
    ],
    "Value": [
        f"{forward_clean_price:.6f}"
    ]
}

df_forward_price = pd.DataFrame(forward_price_data)
df_forward_price

,Description,Value
0,Forward Price,102.372489


### 1.3 The Forward Drop

The **forward drop** is the difference between the spot price and the forward price:

$$
\text{Forward Drop} = P_t - F(t, T)
$$

**Explanation:**  
In our example, the forward price ($102.3725$) is higher than the spot price ($102.0625$). This results in a negative drop (also called a forward premium).

#### Carry Relationship

The financing cost (repo rate of 4.85%) significantly exceeds the bond's current yield accrual (4.00% coupon) over this period.

Mathematically:  
**Cost of Financing $>$ Coupon Income**

To prevent arbitrage, the forward price must be higher, compensating a short position for holding the asset while paying more in financing (repo) interest than is earned from the bond's coupon.

> The trade has **Negative Carry** (Income < Cost), implying a **Positive Cost of Carry** (Cost > Income) in the pricing formula, resulting in a forward price higher than the spot price.

In [4]:
# Forward Drop
forward_drop = spot_price_32nds - forward_clean_price

forward_drop_data = {
    "Description": [
        "Spot Price",
        "Forward Price",
        "Forward Drop (Spot - Forward)"
    ],
    "Value": [
        f"{spot_price_32nds:.6f}",
        f"{forward_clean_price:.6f}",
        f"{forward_drop:.6f}"
    ]
}

df_forward_drop = pd.DataFrame(forward_drop_data)
df_forward_drop

,Description,Value
0,Spot Price,102.062500
1,Forward Price,102.372489
2,Forward Drop (Spot - Forward),-0.309989


## 2. Carry Analysis

### 2.1 Calculate Carry

**Carry** is the net profit or loss from holding the bond position from $t$ to $T$, accounting for coupon income and financing costs.

**Formula:**
$$
\text{Carry} = \text{Coupon Accrual} - \text{Financing Cost}
$$
or, approximately:
$$
\text{Carry} \approx (A_T - A_t) - \left(P_t^{dirty} \times r_{repo} \times \tau_{ACT/360}\right)
$$

where:  
$A_T$ = Accrued interest at delivery date ($T$)  
$A_t$ = Accrued interest at spot date ($t$)  
$P_t^{dirty}$ = Dirty/Full price at time $t$  
$r_{repo}$ = Repo (financing) rate (annualized, ACT/360)  
$\tau_{ACT/360}$ = Year fraction from $t$ to $T$ (Actual/360)

In [6]:
# Calculate Carry
# Income from holding bond (Accrued Interest Gain)
coupon_income = accrued_T - accrued_t

# Cost of financing the position
financing_cost = dirty_price_t * repo_rate * tau_act360

carry = coupon_income - financing_cost

carry_data = {
    "Description": [
        "Coupon Income (A_T - A_t)",
        "Financing Cost",
        "Net Carry"
    ],
    "Value": [
        f"{coupon_income:.6f}",
        f"{financing_cost:.6f}",
        f"{carry:.6f}"
    ]
}

df_carry = pd.DataFrame(carry_data)
df_carry

,Description,Value
0,Coupon Income (A_T - A_t),1.141304
1,Financing Cost,1.451293
2,Net Carry,-0.309989


We verify the relationship:$$P_t - F(t,T) \approx \text{Carry}$$

In [8]:
# Verify Relationship and display in dataframe
verify_data = {
    "Description": [
        "Forward Drop (LHS)",
        "Carry (RHS)",
        "Difference (LHS - RHS)"
    ],
    "Value": [
        f"{forward_drop:.6f}",
        f"{carry:.6f}",
        f"{(forward_drop - carry):.10f}"
    ]
}

df_verify = pd.DataFrame(verify_data)
df_verify

,Description,Value
0,Forward Drop (LHS),-0.309989
1,Carry (RHS),-0.309989
2,Difference (LHS - RHS),0.0000000000


## 3. Forward Pricing with Interim Coupon

### 3.1 Forward Price with Interim Coupon

**Scenario:**

- **New Forward Date ($T_2$):** 2023-10-15  
- **Interim Coupon Date:** 2023-08-31  
- **Coupon Amount:** \$2.00 ($4\% \times 100 / 2$)  

The holder of the bond receives the coupon on Aug 31. In a forward contract, the "tail" price must reflect that the seller receives this coupon and can reinvest it at the repo rate until delivery at $T_2$.

**Formula:**
$$
F(t,T) = \left[ (P_t + A_t)(1 + r \tau_0) - \frac{C*N}{2} \right] (1 + r \tau_1) - A_{T_2}
$$

where:

- $\tau_0$: $t \rightarrow$ Coupon Date  
- $\tau_1$: Coupon Date $\rightarrow T_2$  
- $A_{T_2}$: Accrued interest at $T_2$ (requires looking at the next coupon period: Aug 31, 2023 to Feb 29, 2024).


In [9]:
# Inputs
forward_date_2 = date(2023, 10, 15)
coupon_payment_date = date(2023, 8, 31)
next_coupon_date_2024 = date(2024, 2, 29)

# Time intervals (ACT/360)
days_t_to_coupon = days_between(current_date, coupon_payment_date)
tau_0 = days_t_to_coupon / 360.0

days_coupon_to_T2 = days_between(coupon_payment_date, forward_date_2)
tau_1 = days_coupon_to_T2 / 360.0

# Accrued Interest at T2 (New period: Aug 31 2023 -> Feb 29 2024)
days_in_new_period = days_between(coupon_payment_date, next_coupon_date_2024)
days_accrued_T2 = days_between(coupon_payment_date, forward_date_2)

accrued_T2 = (coupon_rate * face_value / 2) * (days_accrued_T2 / days_in_new_period)

# Forward Price Calculation
coupon_cash = coupon_rate * face_value / 2

# Grow dirty price to coupon date
val_at_coupon_date = dirty_price_t * (1 + repo_rate * tau_0)

# Pay out coupon (reinvest remainder)
val_after_coupon = val_at_coupon_date - coupon_cash

# Grow remainder to T2
val_at_T2 = val_after_coupon * (1 + repo_rate * tau_1)

# Convert to clean price
forward_clean_price_2 = val_at_T2 - accrued_T2

# Display the results
df_section3 = pd.DataFrame({
    "Description": [
        "Tau 0 (t -> Coupon)",
        "Tau 1 (Coupon -> T2)",
        "Accrued Interest at T2",
        "Forward Price with Interim Coupon"
    ],
    "Value": [
        round(tau_0, 6),
        round(tau_1, 6),
        round(accrued_T2, 6),
        round(forward_clean_price_2, 6)
    ]
})

display(df_section3)

,Description,Value
0,Tau 0 (t -> Coupon),0.375000
1,Tau 1 (Coupon -> T2),0.125000
2,Accrued Interest at T2,0.494505
3,Forward Price with Interim Coupon,102.587722


### 3.2 Compare Forward Prices

We compare the correct forward price ($F_{\text{correct}}$) with a "naive" forward price ($F_{\text{naive}}$) calculated as if no coupon were paid (i.e., simply carrying the bond to maturity without accounting for the cash flow event).

**Economic Intuition:**
 
- **Comparison:** $F_{\text{correct}} < F_{\text{naive}}$
- **Why:** The interim coupon is a cash inflow. When you hold the bond (or calculate the forward price based on holding), receiving $\$2.00$ mid-stream reduces the net capital you have tied up in the trade for the second leg of the term ($\tau_1$).
- If you ignored the coupon, you would be modeling a scenario where you finance the full bond price for the entire duration, accumulating higher financing costs.
- Because the bond pays a coupon, the "break-even" future delivery price is lower because the seller has already been partially compensated by the coupon cash flow.

In [12]:
# Naive calculation (Ignoring the coupon cash flow event)
days_total_t_to_T2 = days_between(current_date, forward_date_2)
tau_total = days_total_t_to_T2 / 360.0

# Simply grow dirty price and subtract final accrued
fwd_naive = dirty_price_t * (1 + repo_rate * tau_total) - accrued_T2

# Display the comparison
df_comparison = pd.DataFrame({
    "Description": [
        "Forward (Correct)",
        "Forward (Naive/No-Coupon)",
        "Difference"
    ],
    "Value": [
        round(forward_clean_price_2, 6),
        round(fwd_naive, 6),
        round(forward_clean_price_2 - fwd_naive, 6)
    ]
})

display(df_comparison)

,Description,Value
0,Forward (Correct),102.587722
1,Forward (Naive/No-Coupon),104.588535
2,Difference,-2.000813


## 4. Forward vs Futures